# Imports

In [1]:
import sys
sys.path.insert(1, '../')

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
from IPython.display import display, HTML
import torch
from tqdm.auto import tqdm
from math import sqrt
from torch.utils.data import random_split

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# my imports
from src.data.load_data import load_tokenized_data

## Loading the dataset (???)

In [2]:
global_seed = 1984
transformers.set_seed(global_seed)

model_checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
df = load_tokenized_data(path='../data/raw/filtered.tsv',
                         cache_path='../data/processed/tokenized.tsv',
                         tokenizer=tokenizer, 
                         flatten=True)

In [4]:
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.raw_data = dataframe

        self.texts = dataframe['text'].tolist()
        self.targets = dataframe['toxicity'].tolist()

        self.inputs = []
        for input, target in zip(self.texts, self.targets):
            model_input = { 'input_ids': input, 'labels': target }
            self.inputs.append(model_input)

    def __getitem__(self, idx):
        return self.inputs[idx]

    def __len__(self):
        return len(self.inputs)

## Preprocessing the data
As usual we will need to preprocess data and tokenize it before passing to model

In [5]:
dataset = ToxicDataset(df)

val_ratio = 0.2
train_dataset, val_dataset = random_split(dataset, [1 - val_ratio, val_ratio])

# train_dataset, val_dataset, temp = random_split(dataset, [0.01, 0.05, 0.94])

In [6]:
def collate_batch(batch):
    return tokenizer.pad(batch, return_tensors='pt')

## Fine-tuning the model

In [7]:
# create a model for the pretrained model
t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
encoder = t5_model.encoder

In [8]:
from torch import nn

class RegressorMk1(nn.Module):
    def __init__(self, encoder, h1=128, h2=32):
        super(RegressorMk1, self).__init__()
        self.encoder = encoder
        self.h1 = nn.Linear(512, h1)
        self.h2 = nn.Linear(h1, h2)
        self.h3 = nn.Linear(h2, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        encoded = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

        x1 = nn.functional.relu(self.h1(encoded))
        x2 = nn.functional.relu(self.h2(x1))
        x3 = self.h3(x2)

        return nn.functional.sigmoid(torch.sum(x3[:,:,0] * attention_mask, dim=1))

class Regressor(nn.Module):
    def __init__(self, encoder):
        super(Regressor, self).__init__()
        self.encoder = encoder
        self.h1 = nn.Linear(512, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        encoded = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

        x = self.h1(encoded)

        return nn.functional.sigmoid(torch.sum(x[:,:,0] * attention_mask, dim=1))

In [9]:
model = Regressor(encoder)

In [10]:
from transformers import Trainer, TrainingArguments

mse = nn.MSELoss()
class RegressorTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = mse(outputs, inputs.pop("labels"))
        return (loss, outputs) if return_outputs else loss

In [11]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    mse = (np.array(preds) - np.array(labels)) ** 2 

    return {
        'MSE': mse,
        'RMSE': sqrt(mse)
    }

In [12]:
batch_size = 64
training_args = TrainingArguments(
    "../models/toxic_regressor_output",
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=10,
    save_steps=5000,
    # fp16=True,
    report_to='tensorboard',
)

trainer = RegressorTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_batch,
    #compute_metrics=compute_metrics
)

In [13]:
trainer.train()

  0%|          | 0/144450 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.1603, 'learning_rate': 0.0001993077189338872, 'epoch': 0.03}
{'loss': 0.1194, 'learning_rate': 0.00019861543786777431, 'epoch': 0.07}
{'loss': 0.1084, 'learning_rate': 0.0001979231568016615, 'epoch': 0.1}
{'loss': 0.1023, 'learning_rate': 0.00019723087573554865, 'epoch': 0.14}
{'loss': 0.1002, 'learning_rate': 0.0001965385946694358, 'epoch': 0.17}
{'loss': 0.0954, 'learning_rate': 0.00019584631360332295, 'epoch': 0.21}
{'loss': 0.0914, 'learning_rate': 0.00019515403253721013, 'epoch': 0.24}
{'loss': 0.0901, 'learning_rate': 0.00019446175147109726, 'epoch': 0.28}
{'loss': 0.0883, 'learning_rate': 0.00019376947040498444, 'epoch': 0.31}
{'loss': 0.0872, 'learning_rate': 0.0001930771893388716, 'epoch': 0.35}
{'loss': 0.0863, 'learning_rate': 0.00019238490827275874, 'epoch': 0.38}
{'loss': 0.0854, 'learning_rate': 0.00019169262720664592, 'epoch': 0.42}
{'loss': 0.0833, 'learning_rate': 0.00019100034614053305, 'epoch': 0.45}
{'loss': 0.082, 'learning_rate': 0.00019030806507442023,

  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 0.06895655393600464, 'eval_runtime': 103.0339, 'eval_samples_per_second': 2243.049, 'eval_steps_per_second': 35.056, 'epoch': 1.0}
{'loss': 0.0713, 'learning_rate': 0.0001799238490827276, 'epoch': 1.0}
{'loss': 0.0669, 'learning_rate': 0.00017923156801661475, 'epoch': 1.04}
{'loss': 0.0657, 'learning_rate': 0.0001785392869505019, 'epoch': 1.07}
{'loss': 0.067, 'learning_rate': 0.00017784700588438908, 'epoch': 1.11}
{'loss': 0.0691, 'learning_rate': 0.00017715472481827623, 'epoch': 1.14}
{'loss': 0.0659, 'learning_rate': 0.00017646244375216338, 'epoch': 1.18}
{'loss': 0.0648, 'learning_rate': 0.00017577016268605056, 'epoch': 1.21}
{'loss': 0.0655, 'learning_rate': 0.0001750778816199377, 'epoch': 1.25}
{'loss': 0.0646, 'learning_rate': 0.00017438560055382487, 'epoch': 1.28}
{'loss': 0.0648, 'learning_rate': 0.00017369331948771202, 'epoch': 1.32}
{'loss': 0.0635, 'learning_rate': 0.00017300103842159917, 'epoch': 1.35}
{'loss': 0.0664, 'learning_rate': 0.00017230875735548633,

...

  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 0.05295313149690628, 'eval_runtime': 104.3652, 'eval_samples_per_second': 2214.436, 'eval_steps_per_second': 34.609, 'epoch': 9.0}
{'loss': 0.0298, 'learning_rate': 1.9314641744548288e-05, 'epoch': 9.03}
{'loss': 0.0295, 'learning_rate': 1.8622360678435447e-05, 'epoch': 9.07}
{'loss': 0.0303, 'learning_rate': 1.7930079612322603e-05, 'epoch': 9.1}
{'loss': 0.0287, 'learning_rate': 1.7237798546209763e-05, 'epoch': 9.14}
{'loss': 0.0296, 'learning_rate': 1.654551748009692e-05, 'epoch': 9.17}
{'loss': 0.0292, 'learning_rate': 1.5853236413984078e-05, 'epoch': 9.21}
{'loss': 0.0304, 'learning_rate': 1.5160955347871237e-05, 'epoch': 9.24}
{'loss': 0.0289, 'learning_rate': 1.4468674281758396e-05, 'epoch': 9.28}
{'loss': 0.0298, 'learning_rate': 1.3776393215645552e-05, 'epoch': 9.31}
{'loss': 0.029, 'learning_rate': 1.308411214953271e-05, 'epoch': 9.35}
{'loss': 0.0298, 'learning_rate': 1.2391831083419869e-05, 'epoch': 9.38}
{'loss': 0.0288, 'learning_rate': 1.1699550017307027e-05

  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 0.05349328741431236, 'eval_runtime': 104.8192, 'eval_samples_per_second': 2204.845, 'eval_steps_per_second': 34.459, 'epoch': 10.0}
{'train_runtime': 50535.0806, 'train_samples_per_second': 182.931, 'train_steps_per_second': 2.858, 'train_loss': 0.04704653177099601, 'epoch': 10.0}


TrainOutput(global_step=144450, training_loss=0.04704653177099601, metrics={'train_runtime': 50535.0806, 'train_samples_per_second': 182.931, 'train_steps_per_second': 2.858, 'train_loss': 0.04704653177099601, 'epoch': 10.0})

In [112]:
# saving model
trainer.save_model('../models/last_toxic_regressor')
torch.save(trainer.model.state_dict(), '../models/last_toxic_regressor/model.pt')

In [120]:
model = Regressor(encoder)
weights = torch.load('../models/last_toxic_regressor/model.pt')
model.load_state_dict(weights)
model = model.to('cuda')

In [115]:
def prompt(model, inference_request, tokenizer=tokenizer):
    input = tokenizer(inference_request, return_tensors="pt").to("cuda")
    outputs = model(**input).item()
    print(outputs)

In [116]:
from src.data.load_data import load_data
text_df = load_data('../data/raw/filtered.tsv', flatten=True)
idx = 0

In [123]:
idx = random.randint(0, len(text_df))
ptext = text_df['text'][idx]
print(ptext)
prompt(model, ptext)

A black car wouldn't be pretty.
0.1153540089726448
